# 🦌 Compounds 🧪

Experiments on building graph fragments that can be composed. This is tricky because to
make reusable fragments a new `id` will have to be stamped out for each element. This
notebook introduces the following elements in `ipyelk.elements`:

- `Node` - wrapper for `ElkNode`
- `Port` - wrapper for `ElkPort`
- `Label` - wrapper for `ElkLabel`
- `Edge` - wrapper for `ElkEdge`
- `Partition` - extends node and has some convience functions for building edges

To stamp out `id`s while remembering the originating objects, a `MarkFactory` class can
be instantiated that owns the `Registry` to maintain the mapping.

In [1]:
from IPython.display import JSON
import importnb
import ipywidgets as W
from IPython.display import display

import ipyelk
from ipyelk.loaders import ElementLoader
from ipyelk.elements import Registry, index, Edge, Label, MarkFactory, Node, Port, layout_options as opt

`ipyelk.contrib.library.activity` extends the base `Elements` into a set of marks that
are appropriate for creating Activity Diagrams. These new marks do not have behaviors or
rules that enforce for how they can be connected.

In [2]:
from ipyelk.contrib.library.activity import (
    Activity,
    ActivityDiagram,
    Decision,
    EndActivity,
    Join,
    Merge,
    StartActivity,
)


def activity_app():
    """Utility function for creating a new Elk app suitable for an Activity Diagram"""

    # configure loader
    loader = ElementLoader(
        default_root_opts = {
            opt.Direction.identifier: opt.Direction(value="DOWN").value,
            opt.HierarchyHandling.identifier: opt.HierarchyHandling().value,
        }
    )
    app = ipyelk.Diagram(
        layout={"height": "100%"},
    )
#     toggle = ipyelk.tools.tools.ToggleCollapsedBtn(app=app)
#     fit = ipyelk.tools.tools.FitBtn(app=app)
#     app.toolbar.commands = [fit, toggle]
    return app, loader

## Example Email Activities

Simple representation of processing an email inbox.

In [3]:
def email_activity_example():

    # Building Elements
    act = ActivityDiagram()

    start = StartActivity()
    end = EndActivity()

    open_email = Activity.make("open email")
    delete_email = Activity.make("delete email")
    read_email = Activity.make("read email")
    reply_email = Activity.make("reply")

    j1 = Join()

    m1 = Merge()

    triage = Decision()
    triage.true.labels = [Label(text="is important")]
    triage.false.labels = [Label(text="is junk")]

    response = Decision()
    response.true.labels = [Label(text="yes")]
    response.false.labels = [Label(text="no")]

    # Connect Elements
    act[start:open_email]
    act[open_email : triage.input : "opening"]
    act[triage.false : delete_email]
    act[delete_email:m1]
    act[triage.true : read_email]
    act[read_email : response.input]
    act[response.false : m1]
    act[response.true : reply_email]
    act[reply_email:m1]
    act[m1:end]

    app, loader = activity_app()

    app.source = loader.load(root=act)
    app.view.symbols = act.symbols
    app.style = act.style
    return app, act

In [4]:
if __name__ == "__main__":
    email_act_app, email_activities = email_activity_example()
    display(email_act_app)

Diagram(children=[HTML(value='<style>.styled-widget-140128760747792 .final-state .inner-circle{fill: var(--jp-…

In [5]:
p0.inlet.index.elements

NameError: name 'p0' is not defined

In [6]:
pipes = email_act_app.pipe.pipes
p0 = pipes[0]
p1 = pipes[1]

In [7]:
p0.id_report

IDReport(duplicated={}, null_ids=[])

In [8]:
convert_elkjson(p0.outlet.value.dict())

NameError: name 'convert_elkjson' is not defined

In [9]:
from ipyelk.elements import convert_elkjson
convert_elkjson(p0.outlet.index.root.dict())

Node(id='4c78ac08-871d-4b24-a3a1-619d7f591e0b', labels=[], layoutOptions={'org.eclipse.elk.direction': 'DOWN', 'org.eclipse.elk.hierarchyHandling': 'INCLUDE_CHILDREN'}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=None, key=None, hidden=None), x=None, y=None, width=0.0, height=0.0, ports=[], children=[Node(id='41fb2689-22a6-442a-ab95-c9420b976b99', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=NodeShape(type='node:use', x=None, y=None, width=12.0, height=12.0, use='initial-state'), key=None, hidden=None), x=None, y=None, width=12.0, height=12.0, ports=[], children=[], edges=[]), Node(id='e53de959-749d-4600-bf37-741a2adf4266', labels=[], layoutOptions={'org.eclipse.elk.portConstraints': 'FIXED_SIDE'}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=NodeShape(type='node:diamond', x=None, y=None, width=20.0, height=20.0, use=None), key=None, hidden=None), x=None, y=None, widt

In [10]:
%debug

> <ipython-input-8-89d43e982159>(1)<module>()
----> 1 convert_elkjson(p0.outlet.value.dict())



ipdb>  c


In [ ]:

ipyelk.MarkElementWidget(value=email_activities)

In [ ]:
from ipyelk.contrib.library import activity
from ipyelk.elements import Node
activity.SimpleArrow(
    source=Node(),
    target=Node(),
).dict()

## Example Email Activities

Simple representation of processing an email inbox.

In [7]:
def website_activity_example():
    priority_edge_opts = {
        "org.eclipse.elk.layered.priority.direction": "10",
    }

    # Building Elements
    act = ActivityDiagram()

    start = StartActivity()
    end = EndActivity()

    landing = Activity.make("Landing Page")
    login = Activity.make("Login", container=True)
    enter_creds = Activity.make("Enter Credentials")
    register = Activity.make("Register", container=True)
    registration = Activity.make("Enter Registration Data")
    confirm_email = Activity.make("Receive Confirmation Email")
    confirm = Activity.make("Click Confirmation Link")

    website = Activity.make("Explore Website")

    login.add_child(enter_creds, "cred")
    register.add_child(registration, "registration")
    register.add_child(confirm_email, "confirm_email")
    register.add_child(confirm, "confirm")

    d1 = Decision()
    d1.true.labels = [Label(text="registered")]
    d1.false.labels = [Label(text="not registered")]

    d2 = Decision()
    d2.true.labels = [Label(text="logged in")]
    d2.false.labels = [Label(text="not logged in")]

    response = Decision()
    response.true.labels = [Label(text="yes")]
    response.false.labels = [Label(text="no")]

    # Connecting Elements
    act[start:landing].layoutOptions.update(priority_edge_opts)
    act[landing : d1.input].layoutOptions.update(priority_edge_opts)
    act[d1.true : enter_creds]
    act[d1.false : registration]
    act[registration:confirm_email]
    act[confirm_email:confirm]

    m1 = Merge()
    act[enter_creds:m1]
    act[confirm:m1]
    act[m1 : d2.input]
    act[d2.false : landing]
    act[d2.true : website]

    act[website:end]

    # Creating App and setting the source
    ilk = MarkFactory()
    app, loader = activity_app()
    app.source = loader.load(root=act)
    app.view.symbols = act.symbols
    app.style = act.style
    return app

In [8]:
if __name__ == "__main__":
    website_app = website_activity_example()
    display(website_app)

Diagram(children=[HTML(value='<style>.styled-widget-140128726727504 .final-state .inner-circle{fill: var(--jp-…

In [11]:
website_app.view.selection.ids

('cb2aee52-dfe3-43c5-880d-20e61a524531',)

In [9]:
JSON(website_app.view.source.value.dict())

<IPython.core.display.JSON object>

In [19]:
[p0, p1, p2, *p] = website_app.pipe.pipes
p0.apply_fixes()

AssertionError: Incoming port owned by different node

In [ ]:
%debug

> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/elements/elements.py(203)set_parent()
    201         assert (
    202             self._parent is None or self._parent is parent
--> 203         ), "Incoming port owned by different node"
    204         self._parent = parent
    205         return self



ipdb>  self._parent


Activity(id=None, labels=[Label(id=None, labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_LEFT V_TOP INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClasses='', shape=None, key=None, hidden=None, selectable=False), x=None, y=None, width=None, height=None, text='Register')], layoutOptions={'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE'}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='activity-container', shape=Rect(type='node', x=None, y=None, width=None, height=None, use=None), key=None, hidden=None), x=None, y=None, width=None, height=None, ports=[], children=[Activity(id='db105ee8-8b22-4abd-a1b8-c8cc2f82058c', labels=[Label(id='0ad79a1e-b001-4e95-b580-f2698eee7533', labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_CENTER INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClasses='', shape=None, key=None, hidden=None, selectable=False), x=None,

ipdb>  parent


ActivityDiagram(id='483ee0c2-f9a4-4929-8e4d-17e2307534cf', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=None, key=None, hidden=None), x=None, y=None, width=None, height=None, ports=[], children=[EndActivity(id='6781d485-f9bc-4a42-a2fe-4ea571466701', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=Use(type='node:use', x=None, y=None, width=12.0, height=12.0, use='final-state'), key=None, hidden=None), x=None, y=None, width=None, height=None, ports=[], children=[], edges=[]), StartActivity(id='c00cf9b3-f9eb-4d97-8f3a-0317b7b678b5', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=Use(type='node:use', x=None, y=None, width=12.0, height=12.0, use='initial-state'), key=None, hidden=None), x=None, y=None, width=None, height=None, ports=[], children=[], edges=[]), StartActivity(id='c00cf9b3-f9eb-4d97-8f3a-0317b7b678b5', label

...

'483ee0c2-f9a4-4929-8e4d-17e2307534cf'


ipdb>  self._parent.id
ipdb>  type(self._parent)


<class 'ipyelk.contrib.library.activity.Activity'>


ipdb>  type(parent)


<class 'ipyelk.contrib.library.activity.ActivityDiagram'>


In [ ]:
from ipyelk.elements import iter_elements
iter_elements

In [10]:
website_app.build_index().root()

ActivityDiagram(id=None, labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=None, key=None, hidden=None), x=None, y=None, width=None, height=None, ports=[], children=[], edges=[SimpleArrow(id=None, labels=[], layoutOptions={'org.eclipse.elk.layered.priority.direction': '10'}, metadata=ElementMetadata(), properties=EdgeProperties(cssClasses='', shape=EdgeShape(type='edge', start=None, end='arrow'), key=None, hidden=None), source=StartActivity(id=None, labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=Use(type='node:use', x=None, y=None, width=12.0, height=12.0, use='initial-state'), key=None, hidden=None), x=None, y=None, width=None, height=None, ports=[], children=[], edges=[]), target=Activity(id=None, labels=[Label(id=None, labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_CENTER INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClas

# Record Nodes

Example showing the combination of blocks and activities

In [ ]:
import importnb
import ipywidgets as W
import traitlets as T
from IPython.display import display

import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools
from ipyelk import Elk
from ipyelk.contrib.library.block import Aggregation, Block, BlockDiagram, Composition
from ipyelk.diagram import elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.elements import (
    Compartment,
    Edge,
    Label,
    Mark,
    MarkFactory,
    Node,
    Port,
    Record,
)


class ToggleRecordBtn(ipyelk.tools.tools.ToggleCollapsedBtn):
    def get_related(self, node):
        tree = self.app.transformer.source[1]
        if isinstance(node, Mark) and isinstance(node.element, Compartment):
            parent = list(tree.predecessors(node))[0]
            return [child for i, child in enumerate(tree.neighbors(parent)) if i > 0]
        return super().get_related(node)


def block_app():
    """Utility function for creating a new Elk app suitable for an Activity Diagram"""
    diagram_opts = opt.OptionsWidget(
        options=[opt.Direction(value="RIGHT"), opt.HierarchyHandling()]
    ).value

    # configure app
    app = Elk(
        transformer=ipyelk.nx.XELK(
            layouts={
                elk_model.ElkRoot: {
                    "parents": diagram_opts,
                },
            },
        ),
        layout={"height": "100%"},
    )
    toggle = ToggleRecordBtn(app=app)
    fit = ipyelk.tools.tools.FitBtn(app=app)
    app.toolbar.commands = [fit, toggle]
    return app


def car_example():
    bd = BlockDiagram()

    # Nodes
    vehicle = Block(width=220)
    vehicle.title = Compartment(headings=["Vehicle", "«block»"])
    vehicle.behaviors = Compartment(headings=["Behavior"], content=[" "])

    wheel = Block(width=180)
    wheel.title = Compartment(headings=["Wheel", "«block»"])
    wheel.attrs = Compartment(headings=["properties"], content=["- radius: float"])

    wheel_break = Block()
    wheel_break.title = Compartment(headings=["Break", "«block»"])
    tire = Block()
    tire.title = Compartment(headings=["Tire", "«block»"])

    engine = Block(width=180)
    engine.title = Compartment(headings=["Engine", "«block»"])

    # Edges
    bd[vehicle:wheel:Composition]
    bd[vehicle:engine:Composition]
    bd[wheel:wheel_break:Composition]
    bd[wheel:tire:Composition]

    # internal activities of car
    act = ActivityDiagram().add_class("internal")
    act.start = Activity.make("start engine")
    act.drive = Activity.make("drive")
    act.park = Activity.make("park")
    act[act.start : act.drive]
    act[act.drive : act.park]

    behavior = vehicle.behaviors.add_child(act, "activites")

    # merge defs for both block and activities
    bd.symbols = bd.symbols.merge(act.symbols)
    return bd


def example_car_blocks():
    car = car_example()
    app = block_app()
    cp = MarkFactory()
    app.transformer.source = cp(car)
    app.style = car.style
    app.diagram.symbols = car.symbols

    return app

In [ ]:
if __name__ == "__main__":
    car_app = example_car_blocks()
    display(car_app)